In [18]:
import random
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
fw = open('Transaction-Record.txt','w')

In [19]:
class Customers(object):
    '''
    _smartToken: the token customers want to buy -- SmartToken()
    _ownedSmartToken: smart token's number customers have, being rounded as whitepaper
    _reserveTokens: reserve token's number customers have, being rounded as whitepaper
    _ownedvalue = _reserveValue + _smartValue: the total money cust has, using reserveToken as measurement
    _originalMoney: original money cust has
    _moneyBalance: value of money gained or lost, comparing to the original state
    _expectedPrice: how much money customers are willing to finish the transaction
    '''
    def __init__(self, smartToken, ownedSmartTokens = 0, reserveTokens = 0, expectedPrice = 0):
        self._smartToken = smartToken
        self._ownedSmartTokens = int(ownedSmartTokens)
        self._reserveTokens = int(reserveTokens)
        self._ownedvalue = float(smartToken.getPrice() * self._ownedSmartTokens + self._reserveTokens * 1)
        self._originalMoney = self._ownedvalue
        self._moneyBalance = float(0)
        self._expectedPrice = float(expectedPrice)
        
    def printinfo(self):
        # since new reserve will be converted into or out the smarttoken, Price will change
        # when print info, update the _ownedvalue
        self._ownedvalue = float(self._smartToken.getPrice() * self._ownedSmartTokens + self._reserveTokens * 1)
        self._moneyBalance = self._ownedvalue - self._originalMoney
        print '------'
        print 'smartToken Name:', self._smartToken._Name, '| expected price:', self._expectedPrice
        print 'ownedvalue:', self._ownedvalue, '| money Balance:', self._moneyBalance
    def getReserveTokens(self):
        return self._reserveTokens
    def getownedSmartTokens(self):
        return self._ownedSmartTokens
    def getmoneyBalance(self):
        self._ownedvalue = float(self._smartToken.getPrice() * self._ownedSmartTokens + self._reserveTokens * 1)
        self._moneyBalance = self._ownedvalue - self._originalMoney
        return self._moneyBalance
    def getExpectedPrice(self):
        return self._expectedPrice
    # add customer's reserve amount
    def addReserve(self, addAmount):
        if isinstance(addAmount,int):
            self._reserveTokens = self._reserveTokens + addAmount
            self._ownedvalue = float(self._smartToken.getPrice() * self._ownedSmartTokens + self._reserveTokens * 1)
            # this part money is independant with bancor balance
            self._originalMoney = self._originalMoney + addAmount * 1
        else:
            print "** ERROR, only can add integer # of reserveTokens to reserve"
    # change the expected price
    def changeExpectedPrice(self, newExpectedPrice):
        self._expectedPrice = newExpectedPrice

    '''
    use reserveToken to buy smartToken -> smartToken price increase
    call smartTokens.purchasing() function
    '''
    def buy(self, reserveTokenNumber):
        if reserveTokenNumber < 0:
            print '** ERROR, cannot buy negative number of smartToken'
            return
        if reserveTokenNumber > self._reserveTokens:
            print '** ERROR, invalid Operation in buy'
            return
        if not isinstance(reserveTokenNumber,int):
            print '** ERROR, should use integer number of reserveTokens to buy'
            return
        self._reserveTokens = self._reserveTokens - reserveTokenNumber
        issuedSmartToken = self._smartToken.purchasing(reserveTokenNumber)
        self._ownedSmartTokens = self._ownedSmartTokens + issuedSmartToken
        self._ownedvalue = float(self._smartToken.getPrice() * self._ownedSmartTokens + self._reserveTokens * 1)
        self._moneyBalance = self._ownedvalue - self._originalMoney
        
    '''
    sell smartToken to get reserveToken -> smartToken price decrease
    call smartTokens.destroying() function
    '''        
    def sell(self, smartTokenNumber):
        if smartTokenNumber < 0:
            print '** ERROR, cannot sell negative number of smartToken'
            return
        if self._ownedSmartTokens < smartTokenNumber:
            print '** ERROR, invalid Operation in sell'
            return
        if not isinstance(smartTokenNumber,int):
            print '** ERROR, should use integer number of smartTokens to sell'
            return
        self._ownedSmartTokens = self._ownedSmartTokens - smartTokenNumber
        ReceivedToken = self._smartToken.destroying(smartTokenNumber)
        self._reserveTokens = self._reserveTokens + ReceivedToken
        self._ownedvalue = float(self._smartToken.getPrice() * self._ownedSmartTokens + self._reserveTokens * 1)
        self._moneyBalance = self._ownedvalue - self._originalMoney

In [20]:
class Smartcoin(object):
	'''
	_Name: Name of Smart token
	_ReservetokenName: Name of Reserve Token
	_CRR: CRR
	_Price: the current Price of smart token
	_Supply: the number of issued smart tokens
	_ReserveBalance: the balance of the reserve pool
	'''
	def __init__(self, name='kennycoin', reservetokenName='aCoin', initCRR=0.5, initPrice=1, initIssueNum=100):
		self._Name = name
		self._ReservetokenName = reservetokenName
		self._CRR = float(initCRR)
		self._Price = float(initPrice)
		self._Supply = float(initIssueNum)
		self._ReserveBalance = float(initCRR * initIssueNum)   

	def printInfo(self):
		print '---------'
		print 'NAME:', self._Name, '| RESERVE NAME:', self._ReservetokenName, 'CRR:', self._CRR
		print 'PRICE:',self._Price
		print 'SUPPLY:', self._Supply, '| RESERVE BALANCE:', self._ReserveBalance

	def saveInfo(self, fw):
		fw.write('---------\n')
		fw.write('NAME: '+str(self._Name)+' | RESERVE NAME: '+str(self._ReservetokenName) + ' | CRR: '+str(self._CRR)+'\n')
		fw.write('PRICE: '+str(self._Price))
		fw.write('SUPPLY: '+str(self._Supply)+' | RESERVE BALANCE: '+str(self._ReserveBalance)+'\n')
        
	def updatePrice(self, reserveBalance, supply, CRR):
		newPrice = reserveBalance/(supply * CRR)
		return newPrice
    
	def setCRR(self, newCRR = 0.5):
		oldCRR = self._CRR
		self._CRR = newCRR
		print 'CRR', oldCRR, '->', newCRR

	def getPrice(self):
		return self._Price

	def purchasing(self, convertIntoNum=0):
		# e.g., ETH be convert into BNT, convertInto Num
		issuedtokenNum = round(self._Supply * (((self._ReserveBalance + convertIntoNum)/self._ReserveBalance)**(self._CRR) - 1))
		self._Supply = self._Supply + issuedtokenNum
		self._ReserveBalance = self._ReserveBalance + convertIntoNum
		# update the price after purchasing, e.g. ETH convert into BNT (cust use eth to buy bnt)
		oldPrice = self._Price
		self._Price = self.updatePrice(self._ReserveBalance, self._Supply, self._CRR)
		increaseRatio = (self._Price - oldPrice)/oldPrice
		return int(issuedtokenNum)

	def destroying(self, convertOutNum=0):
		# e.g., BNT be converted out to ETH, convertOutNum is BNT's num
		destroyedtokenNum = convertOutNum
		reserveReceivedNum = round(self._ReserveBalance*(1 - ((self._Supply - convertOutNum)/float(self._Supply))**(1/self._CRR)))
		self._Supply = self._Supply - destroyedtokenNum
		self._ReserveBalance = self._ReserveBalance - reserveReceivedNum
		# update the price after destroying, e.g. BNT convert into ETH (cust sell bnt to get eth)
		oldPrice = self._Price
		self._Price = self.updatePrice(self._ReserveBalance, self._Supply, self._CRR)
		decreaseRatio = (oldPrice-self._Price)/oldPrice
		return int(reserveReceivedNum)

In [21]:
# issue a new smart token
initIssue = 3000000
CRR = 0.2
initTransaction = initIssue * (1-CRR)
KennyCoin = Smartcoin(name='Kenny',reservetokenName='ETH',initCRR=0.2, initPrice=1, initIssueNum=initIssue)

# market initialization
# BancorMarket = Market()

'''
init properties: 
We have #TimeRound round, bouncing happens after every bouncing interval
Totally #custNum  customers come in
In each time round, #tx_round transactions, 
   with original reserve: #custOriginalReserve
   with original smarttokens: #custOriginalSmartTokens
'''
TimeRound = 1000
bouncingInterval = 200
bouncingRange = 1.9
tx_round = 100
custNum = 5000
sigma = 0.1
custOriginalReserve_mu = 200
custOriginalSmartTokens_mu = 200
custOriginalReserve = np.random.normal(custOriginalReserve_mu, 0.1, custNum) # 0.5 is sigma
custOriginalSmartTokens = np.random.normal(custOriginalSmartTokens_mu, 0.1, custNum) # 0.5 is sigma
'''
prevent the init money exhausts the balance, 
here we already set initIssue SmartToken to 300000000,
which actually is hard to exhaust,
also, we should consider the situation that the init issued smarttokens is comparably small,
      or the init smarttokens is held in customers in a large amount
'''
if sum(custOriginalSmartTokens) > initTransaction:
    print '&& WARNING, too many init smart tokens from customers'
    if sum(custOriginalSmartTokens) > initIssue:
        print '** ERROR, too many init smart tokens'
        turndown = 1 + 'klk'

#custmer initialization, first set all their expected Price to be -1
custlist = []
i = 0
while i < custNum:
    Joe = Customers(smartToken = KennyCoin, ownedSmartTokens = int(custOriginalSmartTokens[i]), reserveTokens = int(custOriginalReserve[i]), expectedPrice = -1)
    custlist.append(Joe)
    i = i + 1
# moneyTracker records the 
moneyTracker = []
# PriceTracker records the change of the price
PriceTracker = []
# failureTracker records the failure rate of transactions
failed_rateTracker = []
waitingTracker = []

# tx_list records the transaction order which remains in the market
tx_list = []
j=0
while j < TimeRound:
    buyNum = 0
    sellNum = 0
    CurrentPrice = KennyCoin.getPrice()
    '''
    scan the tx_list, follow the order to try transactions one by one, 
    if succeed, remove the tx element.
    P.S. the element in tx_list is (MARK, cust, tokenNum), 
         when MARK is 1, the cust tries to buy, when MARK is -1, cust tries to sell
    '''
    waitingTx_Num_Old = len(tx_list)
    s = 0
    while s < len(tx_list):
        tx_mark = tx_list[s][0]
        tx_cust = tx_list[s][1]
        tx_tokenNum = tx_list[s][2]
        if tx_mark == - 1:
            # try sell
            if tx_cust.getExpectedPrice() < KennyCoin.getPrice():
                # cust sells successfully
                tx_cust.sell(tx_tokenNum)
                # change expectedPrice, means could do transaction again
                tx_cust.changeExpectedPrice(-1)
                sellNum = sellNum + 1
                tx_list.pop(s)
                s = s - 1
        else:
            # try buy
            if tx_cust.getExpectedPrice() >= KennyCoin.getPrice():
                # cust buys successfully
                tx_cust.buy(tx_tokenNum)
                # change expectedPrice, means could do transaction again
                tx_cust.changeExpectedPrice(-1)
                buyNum = buyNum + 1
                tx_list.pop(s)
                s = s - 1
        s = s + 1

    flag = 0
    if (j > 0) and (j % bouncingInterval == 0):
        # here assume the bouncing_mu is generated by random
        Price_mu = random.uniform(0.1 * CurrentPrice, bouncingRange * CurrentPrice)
        # when Price_mu < than currentPrice, every one wants to sell, else every one wants to buy
        if Price_mu < CurrentPrice:
            flag = -1 # sell
        else:
            flag = 1 # buy
    else:
        Price_mu = CurrentPrice

    custExpectedPrice = np.random.normal(Price_mu, sigma, tx_round)
    i = 0
    for t in xrange(0,custNum):
        if i >= tx_round:
            break
        if custlist[t].getExpectedPrice() >= 0:
            continue
        if custExpectedPrice[i] > 0:
            # avoid newExpectedPrice <= 0
            # in fact newExpectedPrice should not be 0, otherwise will be dead buyer
            newExpectedPrice = custExpectedPrice[i]
        else:
            newExpectedPrice = 0.00001 * KennyCoin.getPrice()

        custlist[t].changeExpectedPrice(newExpectedPrice)
        if flag == -1:
            custSmartToken_Num = custlist[t].getownedSmartTokens()
            randomSell = random.randint(0,custSmartToken_Num)            
            if custlist[t].getExpectedPrice() < KennyCoin.getPrice():
                # cust sells successfully
                custlist[t].sell(randomSell)
                # change expectedPrice, means could do transaction again
                custlist[t].changeExpectedPrice(-1)
                sellNum = sellNum + 1
            else:
                # failed in selling, add cust to tx list, -1 means to sell
                tx_list.append((-1, custlist[t],randomSell))
                # BancorMarket.addSeller(custlist[t])

        elif flag == 1:
            custReserveToken_Num = custlist[t].getReserveTokens()
            randomBuy = random.randint(0, custReserveToken_Num)
            if custlist[t].getExpectedPrice() >= KennyCoin.getPrice():
                # cust buys successfully
                custlist[t].buy(randomBuy)
                custlist[t].changeExpectedPrice(-1)
                buyNum = buyNum + 1
            else:
                # failed in buying, add cust to tx list, 1 means to buy
                tx_list.append((1, custlist[t],randomBuy))
                # BancorMarket.addBuyer(custlist[t])

        else:
            '''
            Here we make an important asumption that customers are willing to finish the transaction,
            as they have already known the current price
            So buyers' expected price should be higher than current price, sellers' should be lower.
            
            In fact we are able to choose buy and sell operation by throwing a coin, 
            but the former situation seems to be more reasonable, 
            and the former market can always perform better than chosing by random. 
            So if we find the former one is flawed, the random's case can only be worse.
            '''
            if custlist[t].getExpectedPrice() >= CurrentPrice:
                # buy
                custReserveToken_Num = custlist[t].getReserveTokens()
                randomBuy = random.randint(0, custReserveToken_Num)
                if custlist[t].getExpectedPrice() >= KennyCoin.getPrice():
                    # cust buy successfully
                    custlist[t].buy(randomBuy)
                    # change expectedPrice for cust, means he could do transaction again
                    custlist[t].changeExpectedPrice(-1)
                    buyNum = buyNum + 1
                else:
                    # failed in buying, add cust to tx list, 1 means to buy
                    tx_list.append((1, custlist[t],randomBuy))
                    # BancorMarket.addBuyer(custlist[t])
            else:
                # sell
                custSmartToken_Num = custlist[t].getownedSmartTokens()
                randomSell = random.randint(0,custSmartToken_Num)            
                if custlist[t].getExpectedPrice() < KennyCoin.getPrice():
                    # cust sells successfully
                    custlist[t].sell(randomSell)
                    # change expectedPrice for cust, means he could do transaction again
                    custlist[t].changeExpectedPrice(-1)
                    sellNum = sellNum + 1
                else:
                    # failed in selling, add cust to tx list, -1 means to sell
                    tx_list.append((-1, custlist[t],randomSell))
                    # BancorMarket.addSeller(custlist[t])
        i = i + 1
    # here we shuffle the cust list to let all customer to get oppotunity to make transactions
    random.shuffle(custlist)
    moneyBalance_all = 0
    for Joe in custlist:
        moneyBalance_all = moneyBalance_all + Joe.getmoneyBalance()

    moneyTracker.append((moneyBalance_all,j))
    failed_rateTracker.append((len(tx_list)/float(waitingTx_Num_Old+tx_round),j))
    waitingTracker.append((len(tx_list),j))
    PriceTracker.append((KennyCoin.getPrice(),j))

    print 'The',j,'round:', buyNum, 'buy tx |', sellNum, 'sell tx |', len(tx_list), 'unfinished tx.'
    fw.write('The '+str(j)+' round: '+str(buyNum)+' buy tx | '+str(sellNum)+' sell tx | '+str(len(tx_list))+' unfinished tx.\n')
    j = j + 1

# draw failure rate
j = 0
failedRate = []
myX_FR = []
while j < TimeRound:
    myX_FR.append(j)
    failedRate.append(failed_rateTracker[j][0])
    j = j + 1
x_FR = np.asarray(myX_FR)
y_FR = np.asarray(failedRate)
plt.plot(x_FR,y_FR,'o-',color = 'navy',alpha = 0.8)
plt.title('Failure Rate Change For All Rounds',fontsize = 25)
plt.xlabel('Round #',fontsize = 15)
plt.ylabel('Failure Rate of Transaction', fontsize = 15)
plt.savefig('Figures/FailureRate.png', bbox_inches='tight')
plt.close()

# draw waiting list change
j = 0
waitingNum = []
myX_W = []
while j < TimeRound:
    myX_W.append(j)
    waitingNum.append(waitingTracker[j][0])
    j = j + 1
x_W = np.asarray(myX_W)
y_W = np.asarray(waitingNum)
plt.plot(x_W,y_W,'o-',color = 'navy',alpha = 0.8)
plt.title('Unfinished Transactions Num For All Rounds',fontsize = 25)
plt.xlabel('Round #',fontsize = 15)
plt.ylabel('Unfinished Transactions Num', fontsize = 15)
plt.savefig('Figures/WaitingNum.png', bbox_inches='tight')
plt.close()

# draw price change figure
j = 0
PriceAllRound = []
myX_P = []
while j < TimeRound:
    PriceAllRound.append(PriceTracker[j][0])
    myX_P.append(j)
    j = j + 1
x_P = np.asarray(myX_P)
y_P = np.asarray(PriceAllRound)
plt.plot(x_P, y_P, 'o-',color = 'navy', alpha = 0.8)
plt.title('Price Change For All Round',fontsize = 25)
plt.xlabel('Round #',fontsize = 15)
plt.ylabel('Price of Smart Token', fontsize = 15)
plt.savefig('Figures/Price_Change.png', bbox_inches='tight')
plt.close()

# draw money change figure
j = 0
moneyChange = []
myX_M = []
while j < TimeRound:
    moneyChange.append(moneyTracker[j][0])
    myX_M.append(j)
    j = j + 1
x_M = np.asarray(myX_M)
y_M = np.asarray(moneyChange)
plt.plot(x_M, y_M, 'o-', color = 'navy', alpha = 0.8)
plt.title('Money Balance Change For All Round',fontsize = 25)
plt.xlabel('Round #',fontsize = 15)
plt.ylabel('Money Balance of All Custs', fontsize = 15)
plt.savefig('Figures/MoneyBalance_Change.png', bbox_inches='tight')
plt.close()

The 0 round: 54 buy tx | 46 sell tx | 0 unfinished tx.
The 1 round: 42 buy tx | 58 sell tx | 0 unfinished tx.
The 2 round: 50 buy tx | 50 sell tx | 0 unfinished tx.
The 3 round: 54 buy tx | 46 sell tx | 0 unfinished tx.
The 4 round: 58 buy tx | 42 sell tx | 0 unfinished tx.
The 5 round: 51 buy tx | 48 sell tx | 1 unfinished tx.
The 6 round: 60 buy tx | 39 sell tx | 2 unfinished tx.
The 7 round: 61 buy tx | 38 sell tx | 3 unfinished tx.
The 8 round: 47 buy tx | 53 sell tx | 3 unfinished tx.
The 9 round: 44 buy tx | 56 sell tx | 3 unfinished tx.
The 10 round: 53 buy tx | 47 sell tx | 3 unfinished tx.
The 11 round: 51 buy tx | 49 sell tx | 3 unfinished tx.
The 12 round: 43 buy tx | 56 sell tx | 4 unfinished tx.
The 13 round: 47 buy tx | 54 sell tx | 3 unfinished tx.
The 14 round: 47 buy tx | 53 sell tx | 3 unfinished tx.
The 15 round: 51 buy tx | 49 sell tx | 3 unfinished tx.
The 16 round: 46 buy tx | 53 sell tx | 4 unfinished tx.
The 17 round: 43 buy tx | 57 sell tx | 4 unfinished tx.
Th

The 151 round: 46 buy tx | 54 sell tx | 3 unfinished tx.
The 152 round: 57 buy tx | 43 sell tx | 3 unfinished tx.
The 153 round: 47 buy tx | 53 sell tx | 3 unfinished tx.
The 154 round: 55 buy tx | 45 sell tx | 3 unfinished tx.
The 155 round: 48 buy tx | 52 sell tx | 3 unfinished tx.
The 156 round: 49 buy tx | 51 sell tx | 3 unfinished tx.
The 157 round: 54 buy tx | 46 sell tx | 3 unfinished tx.
The 158 round: 48 buy tx | 52 sell tx | 3 unfinished tx.
The 159 round: 53 buy tx | 48 sell tx | 2 unfinished tx.
The 160 round: 47 buy tx | 53 sell tx | 2 unfinished tx.
The 161 round: 55 buy tx | 45 sell tx | 2 unfinished tx.
The 162 round: 42 buy tx | 57 sell tx | 3 unfinished tx.
The 163 round: 41 buy tx | 58 sell tx | 4 unfinished tx.
The 164 round: 41 buy tx | 57 sell tx | 6 unfinished tx.
The 165 round: 47 buy tx | 53 sell tx | 6 unfinished tx.
The 166 round: 55 buy tx | 45 sell tx | 6 unfinished tx.
The 167 round: 47 buy tx | 53 sell tx | 6 unfinished tx.
The 168 round: 47 buy tx | 53 s

The 299 round: 46 buy tx | 54 sell tx | 8 unfinished tx.
The 300 round: 51 buy tx | 50 sell tx | 7 unfinished tx.
The 301 round: 49 buy tx | 51 sell tx | 7 unfinished tx.
The 302 round: 48 buy tx | 52 sell tx | 7 unfinished tx.
The 303 round: 49 buy tx | 51 sell tx | 7 unfinished tx.
The 304 round: 35 buy tx | 63 sell tx | 9 unfinished tx.
The 305 round: 51 buy tx | 51 sell tx | 7 unfinished tx.
The 306 round: 45 buy tx | 55 sell tx | 7 unfinished tx.
The 307 round: 50 buy tx | 49 sell tx | 8 unfinished tx.
The 308 round: 53 buy tx | 48 sell tx | 7 unfinished tx.
The 309 round: 51 buy tx | 49 sell tx | 7 unfinished tx.
The 310 round: 53 buy tx | 46 sell tx | 8 unfinished tx.
The 311 round: 44 buy tx | 56 sell tx | 8 unfinished tx.
The 312 round: 49 buy tx | 51 sell tx | 8 unfinished tx.
The 313 round: 52 buy tx | 48 sell tx | 8 unfinished tx.
The 314 round: 49 buy tx | 51 sell tx | 8 unfinished tx.
The 315 round: 49 buy tx | 50 sell tx | 9 unfinished tx.
The 316 round: 43 buy tx | 55 s

The 445 round: 51 buy tx | 49 sell tx | 9 unfinished tx.
The 446 round: 52 buy tx | 49 sell tx | 8 unfinished tx.
The 447 round: 61 buy tx | 38 sell tx | 9 unfinished tx.
The 448 round: 55 buy tx | 46 sell tx | 8 unfinished tx.
The 449 round: 57 buy tx | 43 sell tx | 8 unfinished tx.
The 450 round: 52 buy tx | 47 sell tx | 9 unfinished tx.
The 451 round: 54 buy tx | 46 sell tx | 9 unfinished tx.
The 452 round: 49 buy tx | 51 sell tx | 9 unfinished tx.
The 453 round: 47 buy tx | 53 sell tx | 9 unfinished tx.
The 454 round: 51 buy tx | 49 sell tx | 9 unfinished tx.
The 455 round: 45 buy tx | 54 sell tx | 10 unfinished tx.
The 456 round: 47 buy tx | 53 sell tx | 10 unfinished tx.
The 457 round: 45 buy tx | 56 sell tx | 9 unfinished tx.
The 458 round: 48 buy tx | 52 sell tx | 9 unfinished tx.
The 459 round: 49 buy tx | 52 sell tx | 8 unfinished tx.
The 460 round: 49 buy tx | 51 sell tx | 8 unfinished tx.
The 461 round: 51 buy tx | 49 sell tx | 8 unfinished tx.
The 462 round: 46 buy tx | 54

The 591 round: 45 buy tx | 56 sell tx | 10 unfinished tx.
The 592 round: 50 buy tx | 51 sell tx | 9 unfinished tx.
The 593 round: 51 buy tx | 49 sell tx | 9 unfinished tx.
The 594 round: 50 buy tx | 50 sell tx | 9 unfinished tx.
The 595 round: 44 buy tx | 56 sell tx | 9 unfinished tx.
The 596 round: 49 buy tx | 51 sell tx | 9 unfinished tx.
The 597 round: 51 buy tx | 49 sell tx | 9 unfinished tx.
The 598 round: 49 buy tx | 50 sell tx | 10 unfinished tx.
The 599 round: 54 buy tx | 46 sell tx | 10 unfinished tx.
The 600 round: 100 buy tx | 1 sell tx | 9 unfinished tx.
The 601 round: 48 buy tx | 54 sell tx | 7 unfinished tx.
The 602 round: 52 buy tx | 48 sell tx | 7 unfinished tx.
The 603 round: 41 buy tx | 59 sell tx | 7 unfinished tx.
The 604 round: 43 buy tx | 56 sell tx | 8 unfinished tx.
The 605 round: 47 buy tx | 53 sell tx | 8 unfinished tx.
The 606 round: 44 buy tx | 55 sell tx | 9 unfinished tx.
The 607 round: 48 buy tx | 52 sell tx | 9 unfinished tx.
The 608 round: 54 buy tx | 4

The 736 round: 51 buy tx | 49 sell tx | 5 unfinished tx.
The 737 round: 45 buy tx | 54 sell tx | 6 unfinished tx.
The 738 round: 54 buy tx | 44 sell tx | 8 unfinished tx.
The 739 round: 54 buy tx | 47 sell tx | 7 unfinished tx.
The 740 round: 59 buy tx | 41 sell tx | 7 unfinished tx.
The 741 round: 51 buy tx | 50 sell tx | 6 unfinished tx.
The 742 round: 52 buy tx | 48 sell tx | 6 unfinished tx.
The 743 round: 45 buy tx | 55 sell tx | 6 unfinished tx.
The 744 round: 44 buy tx | 56 sell tx | 6 unfinished tx.
The 745 round: 46 buy tx | 53 sell tx | 7 unfinished tx.
The 746 round: 48 buy tx | 53 sell tx | 6 unfinished tx.
The 747 round: 48 buy tx | 53 sell tx | 5 unfinished tx.
The 748 round: 52 buy tx | 48 sell tx | 5 unfinished tx.
The 749 round: 51 buy tx | 49 sell tx | 5 unfinished tx.
The 750 round: 50 buy tx | 50 sell tx | 5 unfinished tx.
The 751 round: 47 buy tx | 53 sell tx | 5 unfinished tx.
The 752 round: 49 buy tx | 51 sell tx | 5 unfinished tx.
The 753 round: 47 buy tx | 52 s

The 881 round: 52 buy tx | 48 sell tx | 7 unfinished tx.
The 882 round: 44 buy tx | 56 sell tx | 7 unfinished tx.
The 883 round: 49 buy tx | 52 sell tx | 6 unfinished tx.
The 884 round: 45 buy tx | 55 sell tx | 6 unfinished tx.
The 885 round: 45 buy tx | 54 sell tx | 7 unfinished tx.
The 886 round: 49 buy tx | 52 sell tx | 6 unfinished tx.
The 887 round: 45 buy tx | 54 sell tx | 7 unfinished tx.
The 888 round: 40 buy tx | 60 sell tx | 7 unfinished tx.
The 889 round: 53 buy tx | 47 sell tx | 7 unfinished tx.
The 890 round: 46 buy tx | 54 sell tx | 7 unfinished tx.
The 891 round: 57 buy tx | 43 sell tx | 7 unfinished tx.
The 892 round: 50 buy tx | 50 sell tx | 7 unfinished tx.
The 893 round: 41 buy tx | 58 sell tx | 8 unfinished tx.
The 894 round: 47 buy tx | 52 sell tx | 9 unfinished tx.
The 895 round: 48 buy tx | 53 sell tx | 8 unfinished tx.
The 896 round: 50 buy tx | 52 sell tx | 6 unfinished tx.
The 897 round: 52 buy tx | 46 sell tx | 8 unfinished tx.
The 898 round: 51 buy tx | 49 s

In [44]:
'''
here the case is the customers are just sell or buy in random -- throwing a coin to decide sell or buy,
while ignoring the current price might be lower than expected sell price, or higher than expected buy price
'''
# issue a new smart token
initIssue = 3000000
CRR = 0.2
initTransaction = initIssue * (1-CRR)
KennyCoin = Smartcoin(name='Kenny',reservetokenName='ETH',initCRR=0.2, initPrice=1,initIssueNum=initIssue)

# market initialization
# BancorMarket = Market()

'''
init properties: 
We have #TimeRound round, bouncing happens after every bouncing interval
Totally #custNum  customers come in
In each time round, #tx_round transactions, 
   with original reserve: #custOriginalReserve
   with original smarttokens: #custOriginalSmartTokens
'''
TimeRound = 1000
bouncingInterval = 200
bouncingRange = 10
tx_round = 100
custNum = 5000
sigma = 0.1
custOriginalReserve_mu = 200
custOriginalSmartTokens_mu = 200
custOriginalReserve = np.random.normal(custOriginalReserve_mu, 0.1, custNum) # 0.5 is sigma
custOriginalSmartTokens = np.random.normal(custOriginalSmartTokens_mu, 0.1, custNum) # 0.5 is sigma
'''
prevent the init money exhausts the balance, 
here we already set initIssue SmartToken to a large number,
which actually is hard to be exhausted,
also, we should consider the situation that the init issued smarttokens is comparably small,
      or the init smarttokens is held in customers in a large amount
'''
if sum(custOriginalSmartTokens) > initTransaction:
    print '&& WARNING, too many init smart tokens from customers'
    if sum(custOriginalSmartTokens) > initIssue:
        print '** ERROR, too many init smart tokens'
        turndown = 1 + 'klk'

#custmer initialization, first set all their expected Price to be -1
custlist = []
i = 0
while i < custNum:
    Joe = Customers(smartToken = KennyCoin, ownedSmartTokens = int(custOriginalSmartTokens[i]), reserveTokens=int(custOriginalReserve[i]), expectedPrice=-1)
    custlist.append(Joe)
    i = i + 1
# moneyTracker records the money
moneyTracker = []
# PriceTracker records the change of the price
PriceTracker = []
# failureTracker records the failure rate of transactions
failed_rateTracker = []
waitingTracker = []

# tx_list records the transaction order which remains in the market
tx_list = []
j=0
while j < TimeRound:
    buyNum = 0
    sellNum = 0
    CurrentPrice = KennyCoin.getPrice()
    '''
    scan the tx_list, follow the order to try transactions one by one, 
    if succeed, remove the tx element.
    P.S. the element in tx_list is (MARK, cust, tokenNum), 
         when MARK is 1, the cust tries to buy, when MARK is -1, cust tries to sell
    '''
    waitingTx_Num_Old = len(tx_list)
    s = 0
    while s < len(tx_list):
        tx_mark = tx_list[s][0]
        tx_cust = tx_list[s][1]
        tx_tokenNum = tx_list[s][2]
        if tx_mark == - 1:
            # try sell
            if tx_cust.getExpectedPrice() < KennyCoin.getPrice():
                # cust sells successfully
                tx_cust.sell(tx_tokenNum)
                # change expectedPrice, means could do transaction again
                tx_cust.changeExpectedPrice(-1)
                sellNum = sellNum + 1
                tx_list.pop(s)
                s = s - 1
        else:
            # try buy
            if tx_cust.getExpectedPrice() >= KennyCoin.getPrice():
                # cust buys successfully
                tx_cust.buy(tx_tokenNum)
                # change expectedPrice, means could do transaction again
                tx_cust.changeExpectedPrice(-1)
                buyNum = buyNum + 1
                tx_list.pop(s)
                s = s - 1
        s = s + 1

    flag = 0
    if (j > 0) and (j % bouncingInterval == 0):
        # here assume the bouncing_mu is generated by random, range is determined by bouncing range
        Price_mu = random.uniform(0.1 * CurrentPrice, bouncingRange * CurrentPrice)
        # when Price_mu < than currentPrice, every one wants to sell, else every one wants to buy
        if Price_mu < CurrentPrice:
            flag = -1 # sell
        else:
            flag = 1 # buy
    else:
        Price_mu = CurrentPrice

    custExpectedPrice = np.random.normal(Price_mu, sigma, tx_round)
    i = 0
    for t in xrange(0,custNum):
        if i >= tx_round:
            break
        if custlist[t].getExpectedPrice() >= 0:
            continue
        if custExpectedPrice[i] > 0:
            # avoid newExpectedPrice <= 0
            # in fact newExpectedPrice should not be 0, otherwise will be dead buyer
            newExpectedPrice = custExpectedPrice[i]
        else:
            newExpectedPrice = 0.00001 * KennyCoin.getPrice()

        custlist[t].changeExpectedPrice(newExpectedPrice)
        if flag == -1:
            custSmartToken_Num = custlist[t].getownedSmartTokens()
            randomSell = random.randint(0,custSmartToken_Num)            
            if custlist[t].getExpectedPrice() < KennyCoin.getPrice():
                # cust sells successfully
                custlist[t].sell(randomSell)
                # change expectedPrice, means could do transaction again
                custlist[t].changeExpectedPrice(-1)
                sellNum = sellNum + 1
            else:
                # failed in selling, add cust to tx list, -1 means to sell
                tx_list.append((-1, custlist[t],randomSell))
                # BancorMarket.addSeller(custlist[t])

        elif flag == 1:
            custReserveToken_Num = custlist[t].getReserveTokens()
            randomBuy = random.randint(0, custReserveToken_Num)
            if custlist[t].getExpectedPrice() >= KennyCoin.getPrice():
                # cust buys successfully
                custlist[t].buy(randomBuy)
                custlist[t].changeExpectedPrice(-1)
                buyNum = buyNum + 1
            else:
                # failed in buying, add cust to tx list, 1 means to buy
                tx_list.append((1, custlist[t],randomBuy))
                # BancorMarket.addBuyer(custlist[t])

        else:
            '''
            Here customers choose buy and sell operation by throwing a coin, half-half probability 
            '''
            coin = random.randint(0, 100)
            if coin < 50:
                # buy
                custReserveToken_Num = custlist[t].getReserveTokens()
                randomBuy = random.randint(0, custReserveToken_Num)
                if custlist[t].getExpectedPrice() >= KennyCoin.getPrice():
                    # cust buy successfully
                    custlist[t].buy(randomBuy)
                    # change expectedPrice for cust, means he could do transaction again
                    custlist[t].changeExpectedPrice(-1)
                    buyNum = buyNum + 1
                else:
                    # failed in buying, add cust to tx list, 1 means to buy
                    tx_list.append((1, custlist[t],randomBuy))
                    # BancorMarket.addBuyer(custlist[t])
            else:
                # sell
                custSmartToken_Num = custlist[t].getownedSmartTokens()
                randomSell = random.randint(0,custSmartToken_Num)            
                if custlist[t].getExpectedPrice() < KennyCoin.getPrice():
                    # cust sells successfully
                    custlist[t].sell(randomSell)
                    # change expectedPrice for cust, means he could do transaction again
                    custlist[t].changeExpectedPrice(-1)
                    sellNum = sellNum + 1
                else:
                    # failed in selling, add cust to tx list, -1 means to sell
                    tx_list.append((-1, custlist[t],randomSell))
                    # BancorMarket.addSeller(custlist[t])
        i = i + 1
    # here we shuffle the cust list to let all customer to get oppotunity to make transactions
    random.shuffle(custlist)
    moneyBalance_all = 0
    for Joe in custlist:
        moneyBalance_all = moneyBalance_all + Joe.getmoneyBalance()

    moneyTracker.append((moneyBalance_all,j))
    failed_rateTracker.append((len(tx_list)/float(waitingTx_Num_Old+tx_round),j))
    waitingTracker.append((len(tx_list),j))
    PriceTracker.append((KennyCoin.getPrice(),j))

    print 'The',j,'round:', buyNum, 'buy tx |', sellNum, 'sell tx |', len(tx_list), 'unfinished tx.'
    fw.write('The '+str(j)+' round: '+str(buyNum)+' buy tx | '+str(sellNum)+' sell tx | '+str(len(tx_list))+' unfinished tx.\n')
    j = j + 1

# draw failure rate
j = 0
failedRate = []
myX_FR = []
while j < TimeRound:
    myX_FR.append(j)
    failedRate.append(failed_rateTracker[j][0])
    j = j + 1
x_FR = np.asarray(myX_FR)
y_FR = np.asarray(failedRate)
plt.plot(x_FR,y_FR,'o-',color = 'navy',alpha = 0.8)
plt.title('Failure Rate Change For All Rounds',fontsize = 25)
plt.xlabel('Round #',fontsize = 15)
plt.ylabel('Failure Rate of Transaction', fontsize = 15)
plt.savefig('Figures/Coin-FailureRate.png', bbox_inches='tight')
plt.close()

# draw waiting list change
j = 0
waitingNum = []
myX_W = []
while j < TimeRound:
    myX_W.append(j)
    waitingNum.append(waitingTracker[j][0])
    j = j + 1
x_W = np.asarray(myX_W)
y_W = np.asarray(waitingNum)
plt.plot(x_W,y_W,'o-',color = 'navy',alpha = 0.8)
plt.title('Unfinished Transactions Num For All Rounds',fontsize = 25)
plt.xlabel('Round #',fontsize = 15)
plt.ylabel('Unfinished Transactions Num', fontsize = 15)
plt.savefig('Figures/Coin-WaitingNum.png', bbox_inches='tight')
plt.close()

# draw price change figure
j = 0
PriceAllRound = []
myX_P = []
while j < TimeRound:
    PriceAllRound.append(PriceTracker[j][0])
    myX_P.append(j)
    j = j + 1
x_P = np.asarray(myX_P)
y_P = np.asarray(PriceAllRound)
plt.plot(x_P, y_P, 'o-',color = 'navy', alpha = 0.8)
plt.title('Price Change For All Round',fontsize = 25)
plt.xlabel('Round #',fontsize = 15)
plt.ylabel('Price of Smart Token', fontsize = 15)
plt.savefig('Figures/Coin-Price_Change.png', bbox_inches='tight')
plt.close()

# draw money change figure
j = 0
moneyChange = []
myX_M = []
while j < TimeRound:
    moneyChange.append(moneyTracker[j][0])
    myX_M.append(j)
    j = j + 1
x_M = np.asarray(myX_M)
y_M = np.asarray(moneyChange)
plt.plot(x_M, y_M, 'o-', color = 'navy', alpha = 0.8)
plt.title('Money Balance Change For All Round',fontsize = 25)
plt.xlabel('Round #',fontsize = 15)
plt.ylabel('Money Balance of All Custs', fontsize = 15)
plt.savefig('Figures/Coin-MoneyBalance_Change.png', bbox_inches='tight')
plt.close()

The 0 round: 29 buy tx | 20 sell tx | 51 unfinished tx.
The 1 round: 23 buy tx | 25 sell tx | 103 unfinished tx.
The 2 round: 23 buy tx | 32 sell tx | 148 unfinished tx.
The 3 round: 22 buy tx | 31 sell tx | 195 unfinished tx.
The 4 round: 21 buy tx | 22 sell tx | 252 unfinished tx.
The 5 round: 27 buy tx | 21 sell tx | 304 unfinished tx.
The 6 round: 20 buy tx | 24 sell tx | 360 unfinished tx.
The 7 round: 20 buy tx | 22 sell tx | 418 unfinished tx.
The 8 round: 26 buy tx | 20 sell tx | 472 unfinished tx.
The 9 round: 26 buy tx | 19 sell tx | 527 unfinished tx.
The 10 round: 28 buy tx | 27 sell tx | 572 unfinished tx.
The 11 round: 24 buy tx | 14 sell tx | 634 unfinished tx.
The 12 round: 33 buy tx | 25 sell tx | 676 unfinished tx.
The 13 round: 22 buy tx | 24 sell tx | 730 unfinished tx.
The 14 round: 27 buy tx | 27 sell tx | 776 unfinished tx.
The 15 round: 19 buy tx | 16 sell tx | 841 unfinished tx.
The 16 round: 27 buy tx | 26 sell tx | 888 unfinished tx.
The 17 round: 24 buy tx |

The 140 round: 0 buy tx | 0 sell tx | 5000 unfinished tx.
The 141 round: 0 buy tx | 0 sell tx | 5000 unfinished tx.
The 142 round: 0 buy tx | 0 sell tx | 5000 unfinished tx.
The 143 round: 0 buy tx | 0 sell tx | 5000 unfinished tx.
The 144 round: 0 buy tx | 0 sell tx | 5000 unfinished tx.
The 145 round: 0 buy tx | 0 sell tx | 5000 unfinished tx.
The 146 round: 0 buy tx | 0 sell tx | 5000 unfinished tx.
The 147 round: 0 buy tx | 0 sell tx | 5000 unfinished tx.
The 148 round: 0 buy tx | 0 sell tx | 5000 unfinished tx.
The 149 round: 0 buy tx | 0 sell tx | 5000 unfinished tx.
The 150 round: 0 buy tx | 0 sell tx | 5000 unfinished tx.
The 151 round: 0 buy tx | 0 sell tx | 5000 unfinished tx.
The 152 round: 0 buy tx | 0 sell tx | 5000 unfinished tx.
The 153 round: 0 buy tx | 0 sell tx | 5000 unfinished tx.
The 154 round: 0 buy tx | 0 sell tx | 5000 unfinished tx.
The 155 round: 0 buy tx | 0 sell tx | 5000 unfinished tx.
The 156 round: 0 buy tx | 0 sell tx | 5000 unfinished tx.
The 157 round:

The 282 round: 0 buy tx | 0 sell tx | 5000 unfinished tx.
The 283 round: 0 buy tx | 0 sell tx | 5000 unfinished tx.
The 284 round: 0 buy tx | 0 sell tx | 5000 unfinished tx.
The 285 round: 0 buy tx | 0 sell tx | 5000 unfinished tx.
The 286 round: 0 buy tx | 0 sell tx | 5000 unfinished tx.
The 287 round: 0 buy tx | 0 sell tx | 5000 unfinished tx.
The 288 round: 0 buy tx | 0 sell tx | 5000 unfinished tx.
The 289 round: 0 buy tx | 0 sell tx | 5000 unfinished tx.
The 290 round: 0 buy tx | 0 sell tx | 5000 unfinished tx.
The 291 round: 0 buy tx | 0 sell tx | 5000 unfinished tx.
The 292 round: 0 buy tx | 0 sell tx | 5000 unfinished tx.
The 293 round: 0 buy tx | 0 sell tx | 5000 unfinished tx.
The 294 round: 0 buy tx | 0 sell tx | 5000 unfinished tx.
The 295 round: 0 buy tx | 0 sell tx | 5000 unfinished tx.
The 296 round: 0 buy tx | 0 sell tx | 5000 unfinished tx.
The 297 round: 0 buy tx | 0 sell tx | 5000 unfinished tx.
The 298 round: 0 buy tx | 0 sell tx | 5000 unfinished tx.
The 299 round:

The 426 round: 0 buy tx | 0 sell tx | 5000 unfinished tx.
The 427 round: 0 buy tx | 0 sell tx | 5000 unfinished tx.
The 428 round: 0 buy tx | 0 sell tx | 5000 unfinished tx.
The 429 round: 0 buy tx | 0 sell tx | 5000 unfinished tx.
The 430 round: 0 buy tx | 0 sell tx | 5000 unfinished tx.
The 431 round: 0 buy tx | 0 sell tx | 5000 unfinished tx.
The 432 round: 0 buy tx | 0 sell tx | 5000 unfinished tx.
The 433 round: 0 buy tx | 0 sell tx | 5000 unfinished tx.
The 434 round: 0 buy tx | 0 sell tx | 5000 unfinished tx.
The 435 round: 0 buy tx | 0 sell tx | 5000 unfinished tx.
The 436 round: 0 buy tx | 0 sell tx | 5000 unfinished tx.
The 437 round: 0 buy tx | 0 sell tx | 5000 unfinished tx.
The 438 round: 0 buy tx | 0 sell tx | 5000 unfinished tx.
The 439 round: 0 buy tx | 0 sell tx | 5000 unfinished tx.
The 440 round: 0 buy tx | 0 sell tx | 5000 unfinished tx.
The 441 round: 0 buy tx | 0 sell tx | 5000 unfinished tx.
The 442 round: 0 buy tx | 0 sell tx | 5000 unfinished tx.
The 443 round:

The 570 round: 0 buy tx | 0 sell tx | 5000 unfinished tx.
The 571 round: 0 buy tx | 0 sell tx | 5000 unfinished tx.
The 572 round: 0 buy tx | 0 sell tx | 5000 unfinished tx.
The 573 round: 0 buy tx | 0 sell tx | 5000 unfinished tx.
The 574 round: 0 buy tx | 0 sell tx | 5000 unfinished tx.
The 575 round: 0 buy tx | 0 sell tx | 5000 unfinished tx.
The 576 round: 0 buy tx | 0 sell tx | 5000 unfinished tx.
The 577 round: 0 buy tx | 0 sell tx | 5000 unfinished tx.
The 578 round: 0 buy tx | 0 sell tx | 5000 unfinished tx.
The 579 round: 0 buy tx | 0 sell tx | 5000 unfinished tx.
The 580 round: 0 buy tx | 0 sell tx | 5000 unfinished tx.
The 581 round: 0 buy tx | 0 sell tx | 5000 unfinished tx.
The 582 round: 0 buy tx | 0 sell tx | 5000 unfinished tx.
The 583 round: 0 buy tx | 0 sell tx | 5000 unfinished tx.
The 584 round: 0 buy tx | 0 sell tx | 5000 unfinished tx.
The 585 round: 0 buy tx | 0 sell tx | 5000 unfinished tx.
The 586 round: 0 buy tx | 0 sell tx | 5000 unfinished tx.
The 587 round:

The 716 round: 0 buy tx | 0 sell tx | 5000 unfinished tx.
The 717 round: 0 buy tx | 0 sell tx | 5000 unfinished tx.
The 718 round: 0 buy tx | 0 sell tx | 5000 unfinished tx.
The 719 round: 0 buy tx | 0 sell tx | 5000 unfinished tx.
The 720 round: 0 buy tx | 0 sell tx | 5000 unfinished tx.
The 721 round: 0 buy tx | 0 sell tx | 5000 unfinished tx.
The 722 round: 0 buy tx | 0 sell tx | 5000 unfinished tx.
The 723 round: 0 buy tx | 0 sell tx | 5000 unfinished tx.
The 724 round: 0 buy tx | 0 sell tx | 5000 unfinished tx.
The 725 round: 0 buy tx | 0 sell tx | 5000 unfinished tx.
The 726 round: 0 buy tx | 0 sell tx | 5000 unfinished tx.
The 727 round: 0 buy tx | 0 sell tx | 5000 unfinished tx.
The 728 round: 0 buy tx | 0 sell tx | 5000 unfinished tx.
The 729 round: 0 buy tx | 0 sell tx | 5000 unfinished tx.
The 730 round: 0 buy tx | 0 sell tx | 5000 unfinished tx.
The 731 round: 0 buy tx | 0 sell tx | 5000 unfinished tx.
The 732 round: 0 buy tx | 0 sell tx | 5000 unfinished tx.
The 733 round:

The 860 round: 0 buy tx | 0 sell tx | 5000 unfinished tx.
The 861 round: 0 buy tx | 0 sell tx | 5000 unfinished tx.
The 862 round: 0 buy tx | 0 sell tx | 5000 unfinished tx.
The 863 round: 0 buy tx | 0 sell tx | 5000 unfinished tx.
The 864 round: 0 buy tx | 0 sell tx | 5000 unfinished tx.
The 865 round: 0 buy tx | 0 sell tx | 5000 unfinished tx.
The 866 round: 0 buy tx | 0 sell tx | 5000 unfinished tx.
The 867 round: 0 buy tx | 0 sell tx | 5000 unfinished tx.
The 868 round: 0 buy tx | 0 sell tx | 5000 unfinished tx.
The 869 round: 0 buy tx | 0 sell tx | 5000 unfinished tx.
The 870 round: 0 buy tx | 0 sell tx | 5000 unfinished tx.
The 871 round: 0 buy tx | 0 sell tx | 5000 unfinished tx.
The 872 round: 0 buy tx | 0 sell tx | 5000 unfinished tx.
The 873 round: 0 buy tx | 0 sell tx | 5000 unfinished tx.
The 874 round: 0 buy tx | 0 sell tx | 5000 unfinished tx.
The 875 round: 0 buy tx | 0 sell tx | 5000 unfinished tx.
The 876 round: 0 buy tx | 0 sell tx | 5000 unfinished tx.
The 877 round:

In [17]:
# example of using smartcoin class
BNTCoin = Smartcoin(name='BNT',reservetokenName='ETH',initCRR=0.2, initPrice=1,initIssueNum=300000)
print BNTCoin.destroying(10)
BNTCoin.printInfo()
# print BNTCoin.purchasing(300)
# print BNTCoin.purchasing(700)
# BNTCoin.printInfo()
print BNTCoin.destroying(177)
BNTCoin.printInfo()

10
---------
NAME: BNT | RESERVE NAME: ETH CRR: 0.2
PRICE: 0.999866662222
SUPPLY: 299990.0 | RESERVE BALANCE: 59990.0
177
---------
NAME: BNT | RESERVE NAME: ETH CRR: 0.2
PRICE: 0.99750511152
SUPPLY: 299813.0 | RESERVE BALANCE: 59813.0


In [20]:
KennyCoin = Smartcoin(name='Kenny',reservetokenName='ETH',initCRR=0.2, initPrice=1,initIssueNum=300000)
# test for customers class
Alice = Customers(smartToken=KennyCoin,ownedSmartTokens=100,reserveTokens=100,expectedPrice= 2)
Alice.printinfo()
Alice.buy(-1)
Alice.printinfo()
Alice.printinfo()

------
smartToken Name: Kenny | expected price: 2.0
ownedvalue: 200.0 | money Balance: 0.0
** ERROR, cannot buy negative number of smartToken
------
smartToken Name: Kenny | expected price: 2.0
ownedvalue: 200.0 | money Balance: 0.0
------
smartToken Name: Kenny | expected price: 2.0
ownedvalue: 200.0 | money Balance: 0.0
